# Simulating Constitutive Processes of semantic change within heterogeneous populations of speakers

In [1]:
# basic imports
import torch
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

# project code imports
from mod.one_hot_agent import *
from mod.plot import *
from mod.network import *

##### Hyper parameters

In [2]:
no_agents = 20
no_connections = 5
add_vocab_in = 50
semantic_features = 3
starting_observations = 5
starting_uncertainty = .2
words_per_semantic_feature = 5
new_environment_prob = .25
vocab_dropout = .2
enforce_word_feature_mapping = True

## Simple dyadic interaction across repeated turns in a random environment

In [3]:
starting_env = torch.distributions.MultivariateNormal(torch.randn(size=(1,semantic_features)), covariance_matrix=torch.eye(semantic_features) * .2)

In [4]:
net = social_network(
    k_agents=no_agents,
    vocab_size=words_per_semantic_feature,
    connections_per_agent=no_connections,
    semantic_dimensions=semantic_features,
    starting_observations=starting_observations,
    starting_uncertainty=starting_uncertainty
)
net.agents = [create_ablated_vocabulary(a,vocab_dropout) for a in net.agents]

In [5]:
fig = heatmap(
    net.similarity_heatmap(),
    x_labels=['agent_'+str(i+1) for i in range(len(net.agents))],
    y_labels=['agent_'+str(i+1) for i in range(len(net.agents))]
)
fig.show()

In [6]:
# and set the number of iterations
turns = 300

In [7]:
vocab_dif = []

In [8]:
for rd in tqdm(range(turns)):
    env = starting_env.sample()

    net.interaction(env, 3)
    vocab_dif += [{'agent':i + 1, 'interaction': rd, 'Δ P(w|m)':val} for i,val in enumerate(net.similarity_heatmap().mean(dim=-1).tolist())]

vocab_dif = pd.DataFrame(vocab_dif)

  0%|          | 0/300 [00:00<?, ?it/s]

In [9]:
fig = heatmap(
    net.similarity_heatmap(),
    x_labels=['agent_'+str(i+1) for i in range(len(net.agents))],
    y_labels=['agent_'+str(i+1) for i in range(len(net.agents))]
)
fig.show()

In [10]:
fig = multi_agent_update_plot_from_df(vocab_dif)
fig.update_layout(
    # title='Dyadic interaction in a random environment',
    yaxis_title='Δ P(w|m)',
    xaxis_title='interactions'
)
fig.show()

## Random environment and introduction of new terms

In [11]:
starting_env = torch.distributions.MultivariateNormal(torch.randn(size=(1,semantic_features)), covariance_matrix=torch.eye(semantic_features) * .2)

In [12]:
net = social_network(
    k_agents=no_agents,
    vocab_size=words_per_semantic_feature,
    connections_per_agent=no_connections,
    semantic_dimensions=semantic_features,
    starting_observations=starting_observations,
    starting_uncertainty=starting_uncertainty
)
net.agents = [create_ablated_vocabulary(a,vocab_dropout) for a in net.agents]

In [13]:
fig = heatmap(
    net.similarity_heatmap(),
    x_labels=['agent_'+str(i+1) for i in range(len(net.agents))],
    y_labels=['agent_'+str(i+1) for i in range(len(net.agents))]
)
fig.show()

In [14]:
# and set the number of iterations
turns = 300

In [15]:
vocab_dif = []

In [16]:
for rd in tqdm(range(turns)):
    env = starting_env.sample()

    new_vocab_round = ((rd % add_vocab_in) == 0) & (rd != 0)
    net.interaction(env, 3, new_vocab_round)

    vocab_dif += [{'agent':i + 1, 'interaction': rd, 'Δ P(w|m)':val} for i,val in enumerate(net.similarity_heatmap().mean(dim=-1).tolist())]

vocab_dif = pd.DataFrame(vocab_dif)

  0%|          | 0/300 [00:00<?, ?it/s]

In [17]:
fig = heatmap(
    net.similarity_heatmap(),
    x_labels=['agent_'+str(i+1) for i in range(len(net.agents))],
    y_labels=['agent_'+str(i+1) for i in range(len(net.agents))]
)
fig.show()

In [18]:
fig = multi_agent_update_plot_from_df(vocab_dif)
fig.update_layout(
    # title='Dyadic interaction in a random environment',
    yaxis_title='Δ P(w|m)',
    xaxis_title='interactions'
)
fig.show()

## Changing environment

In [19]:
starting_env = torch.distributions.MultivariateNormal(torch.randn(size=(1,semantic_features)), covariance_matrix=torch.eye(semantic_features) * .2)

In [20]:
net = social_network(
    k_agents=no_agents,
    vocab_size=words_per_semantic_feature,
    connections_per_agent=no_connections,
    semantic_dimensions=semantic_features,
    starting_observations=starting_observations,
    starting_uncertainty=starting_uncertainty
)
net.agents = [create_ablated_vocabulary(a,vocab_dropout) for a in net.agents]

In [21]:
fig = heatmap(
    net.similarity_heatmap(),
    x_labels=['agent_'+str(i+1) for i in range(len(net.agents))],
    y_labels=['agent_'+str(i+1) for i in range(len(net.agents))]
)
fig.show()

In [22]:
# and set the number of iterations
turns = 300

In [23]:
vocab_dif = []

In [24]:
for rd in tqdm(range(turns)):

    new_env_prob = torch.rand(size=(1,))
    if new_env_prob > new_environment_prob:
        starting_env = torch.distributions.MultivariateNormal(torch.randn(size=(1,semantic_features)), covariance_matrix=torch.eye(semantic_features) * .2)

    env = starting_env.sample()

    new_vocab_round = False
    net.interaction(env, 3, new_vocab_round)

    vocab_dif += [{'agent':i + 1, 'interaction': rd, 'Δ P(w|m)':val} for i,val in enumerate(net.similarity_heatmap().mean(dim=-1).tolist())]

vocab_dif = pd.DataFrame(vocab_dif)

  0%|          | 0/300 [00:00<?, ?it/s]

In [25]:
fig = heatmap(
    net.similarity_heatmap(),
    x_labels=['agent_'+str(i+1) for i in range(len(net.agents))],
    y_labels=['agent_'+str(i+1) for i in range(len(net.agents))]
)
fig.show()

In [26]:
fig = multi_agent_update_plot_from_df(vocab_dif)
fig.update_layout(
    # title='Dyadic interaction in a random environment',
    yaxis_title='Δ P(w|m)',
    xaxis_title='interactions'
)
fig.show()

## Changing environment and introduction of new terms

In [27]:
starting_env = torch.distributions.MultivariateNormal(torch.randn(size=(1,semantic_features)), covariance_matrix=torch.eye(semantic_features) * .2)

In [28]:
net = social_network(
    k_agents=no_agents,
    vocab_size=words_per_semantic_feature,
    connections_per_agent=no_connections,
    semantic_dimensions=semantic_features,
    starting_observations=starting_observations,
    starting_uncertainty=starting_uncertainty
)
net.agents = [create_ablated_vocabulary(a,vocab_dropout) for a in net.agents]

In [29]:
fig = heatmap(
    net.similarity_heatmap(),
    x_labels=['agent_'+str(i+1) for i in range(len(net.agents))],
    y_labels=['agent_'+str(i+1) for i in range(len(net.agents))]
)
fig.show()

In [30]:
# and set the number of iterations
turns = 300

In [31]:
vocab_dif = []

In [32]:
for rd in tqdm(range(turns)):

    new_env_prob = torch.rand(size=(1,))
    if new_env_prob > new_environment_prob:
        starting_env = torch.distributions.MultivariateNormal(torch.randn(size=(1,semantic_features)), covariance_matrix=torch.eye(semantic_features) * .2)

    env = starting_env.sample()

    new_vocab_round = ((rd % add_vocab_in) == 0) & (rd != 0)
    net.interaction(env, 3, new_vocab_round)

    vocab_dif += [{'agent':i + 1, 'interaction': rd, 'Δ P(w|m)':val} for i,val in enumerate(net.similarity_heatmap().mean(dim=-1).tolist())]

vocab_dif = pd.DataFrame(vocab_dif)

  0%|          | 0/300 [00:00<?, ?it/s]

In [33]:
fig = heatmap(
    net.similarity_heatmap(),
    x_labels=['agent_'+str(i+1) for i in range(len(net.agents))],
    y_labels=['agent_'+str(i+1) for i in range(len(net.agents))]
)
fig.show()

In [34]:
fig = multi_agent_update_plot_from_df(vocab_dif)
fig.update_layout(
    # title='Dyadic interaction in a random environment',
    yaxis_title='Δ P(w|m)',
    xaxis_title='interactions'
)
fig.show()

## Stochastic environment

In [35]:
starting_env = torch.distributions.MultivariateNormal(torch.randn(size=(1,semantic_features)), covariance_matrix=torch.eye(semantic_features) * .2)

In [36]:
net = social_network(
    k_agents=no_agents,
    vocab_size=words_per_semantic_feature,
    connections_per_agent=no_connections,
    semantic_dimensions=semantic_features,
    starting_observations=starting_observations,
    starting_uncertainty=starting_uncertainty
)
net.agents = [create_ablated_vocabulary(a,vocab_dropout) for a in net.agents]

In [37]:
fig = heatmap(
    net.similarity_heatmap(),
    x_labels=['agent_'+str(i+1) for i in range(len(net.agents))],
    y_labels=['agent_'+str(i+1) for i in range(len(net.agents))]
)
fig.show()

In [38]:
# and set the number of iterations
turns = 300

In [39]:
vocab_dif = []

In [40]:
for rd in tqdm(range(turns)):

    env = starting_env.sample()
    starting_env.loc = env

    new_vocab_round = False
    net.interaction(env, 3, new_vocab_round)

    vocab_dif += [{'agent':i + 1, 'interaction': rd, 'Δ P(w|m)':val} for i,val in enumerate(net.similarity_heatmap().mean(dim=-1).tolist())]

vocab_dif = pd.DataFrame(vocab_dif)

  0%|          | 0/300 [00:00<?, ?it/s]

In [41]:
fig = heatmap(
    net.similarity_heatmap(),
    x_labels=['agent_'+str(i+1) for i in range(len(net.agents))],
    y_labels=['agent_'+str(i+1) for i in range(len(net.agents))]
)
fig.show()

In [42]:
fig = multi_agent_update_plot_from_df(vocab_dif)
fig.update_layout(
    # title='Dyadic interaction in a random environment',
    yaxis_title='Δ P(w|m)',
    xaxis_title='interactions'
)
fig.show()

## Stochastic environment and introduction of new terms

In [43]:
starting_env = torch.distributions.MultivariateNormal(torch.randn(size=(1,semantic_features)), covariance_matrix=torch.eye(semantic_features) * .2)

In [44]:
net = social_network(
    k_agents=no_agents,
    vocab_size=words_per_semantic_feature,
    connections_per_agent=no_connections,
    semantic_dimensions=semantic_features,
    starting_observations=starting_observations,
    starting_uncertainty=starting_uncertainty
)
net.agents = [create_ablated_vocabulary(a,vocab_dropout) for a in net.agents]

In [45]:
fig = heatmap(
    net.similarity_heatmap(),
    x_labels=['agent_'+str(i+1) for i in range(len(net.agents))],
    y_labels=['agent_'+str(i+1) for i in range(len(net.agents))]
)
fig.show()

In [46]:
# and set the number of iterations
turns = 300

In [47]:
vocab_dif = []

In [48]:
for rd in tqdm(range(turns)):

    env = starting_env.sample()
    starting_env.loc = env

    new_vocab_round = ((rd % add_vocab_in) == 0) & (rd != 0)
    net.interaction(env, 3, new_vocab_round)

    vocab_dif += [{'agent':i + 1, 'interaction': rd, 'Δ P(w|m)':val} for i,val in enumerate(net.similarity_heatmap().mean(dim=-1).tolist())]

vocab_dif = pd.DataFrame(vocab_dif)

  0%|          | 0/300 [00:00<?, ?it/s]

In [49]:
fig = heatmap(
    net.similarity_heatmap(),
    x_labels=['agent_'+str(i+1) for i in range(len(net.agents))],
    y_labels=['agent_'+str(i+1) for i in range(len(net.agents))]
)
fig.show()

In [50]:
fig = multi_agent_update_plot_from_df(vocab_dif)
fig.update_layout(
    # title='Dyadic interaction in a random environment',
    yaxis_title='Δ P(w|m)',
    xaxis_title='interactions'
)
fig.show()

## Sudden and stochastic environment

In [51]:
starting_env = torch.distributions.MultivariateNormal(torch.randn(size=(1,semantic_features)), covariance_matrix=torch.eye(semantic_features) * .2)

In [52]:
net = social_network(
    k_agents=no_agents,
    vocab_size=words_per_semantic_feature,
    connections_per_agent=no_connections,
    semantic_dimensions=semantic_features,
    starting_observations=starting_observations,
    starting_uncertainty=starting_uncertainty
)
net.agents = [create_ablated_vocabulary(a,vocab_dropout) for a in net.agents]

In [53]:
fig = heatmap(
    net.similarity_heatmap(),
    x_labels=['agent_'+str(i+1) for i in range(len(net.agents))],
    y_labels=['agent_'+str(i+1) for i in range(len(net.agents))]
)
fig.show()

In [54]:
# and set the number of iterations
turns = 300

In [55]:
vocab_dif = []

In [56]:
for rd in tqdm(range(turns)):

    new_env_prob = torch.rand(size=(1,))
    if new_env_prob > new_environment_prob:
        starting_env = torch.distributions.MultivariateNormal(torch.randn(size=(1,semantic_features)), covariance_matrix=torch.eye(semantic_features) * .2)

    env = starting_env.sample()
    starting_env.loc = env

    new_vocab_round = False
    net.interaction(env, 3, new_vocab_round)

    vocab_dif += [{'agent':i + 1, 'interaction': rd, 'Δ P(w|m)':val} for i,val in enumerate(net.similarity_heatmap().mean(dim=-1).tolist())]

vocab_dif = pd.DataFrame(vocab_dif)

  0%|          | 0/300 [00:00<?, ?it/s]

In [57]:
fig = heatmap(
    net.similarity_heatmap(),
    x_labels=['agent_'+str(i+1) for i in range(len(net.agents))],
    y_labels=['agent_'+str(i+1) for i in range(len(net.agents))]
)
fig.show()

In [58]:
fig = multi_agent_update_plot_from_df(vocab_dif)
fig.update_layout(
    # title='Dyadic interaction in a random environment',
    yaxis_title='Δ P(w|m)',
    xaxis_title='interactions'
)
fig.show()

## Sudden and stochastic environment, plus introduction of new terms

In [59]:
starting_env = torch.distributions.MultivariateNormal(torch.randn(size=(1,semantic_features)), covariance_matrix=torch.eye(semantic_features) * .2)

In [60]:
net = social_network(
    k_agents=no_agents,
    vocab_size=words_per_semantic_feature,
    connections_per_agent=no_connections,
    semantic_dimensions=semantic_features,
    starting_observations=starting_observations,
    starting_uncertainty=starting_uncertainty
)
net.agents = [create_ablated_vocabulary(a,vocab_dropout) for a in net.agents]

In [61]:
fig = heatmap(
    net.similarity_heatmap(),
    x_labels=['agent_'+str(i+1) for i in range(len(net.agents))],
    y_labels=['agent_'+str(i+1) for i in range(len(net.agents))]
)
fig.show()

In [62]:
# and set the number of iterations
turns = 300

In [63]:
vocab_dif = []

In [64]:
for rd in tqdm(range(turns)):

    new_env_prob = torch.rand(size=(1,))
    if new_env_prob > new_environment_prob:
        starting_env = torch.distributions.MultivariateNormal(torch.randn(size=(1,semantic_features)), covariance_matrix=torch.eye(semantic_features) * .2)

    env = starting_env.sample()
    starting_env.loc = env

    new_vocab_round = ((rd % add_vocab_in) == 0) & (rd != 0)
    net.interaction(env, 3, new_vocab_round)

    vocab_dif += [{'agent':i + 1, 'interaction': rd, 'Δ P(w|m)':val} for i,val in enumerate(net.similarity_heatmap().mean(dim=-1).tolist())]

vocab_dif = pd.DataFrame(vocab_dif)

  0%|          | 0/300 [00:00<?, ?it/s]

In [65]:
fig = heatmap(
    net.similarity_heatmap(),
    x_labels=['agent_'+str(i+1) for i in range(len(net.agents))],
    y_labels=['agent_'+str(i+1) for i in range(len(net.agents))]
)
fig.show()

In [66]:
fig = multi_agent_update_plot_from_df(vocab_dif)
fig.update_layout(
    # title='Dyadic interaction in a random environment',
    yaxis_title='Δ P(w|m)',
    xaxis_title='interactions'
)
fig.show()